In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
df_5mix = load_excel('/Users/xyuan/Documents/SeqDataSets/Ning1024/5mix_MFE.xlsx', 0)
df_2mix = load_data('/Users/xyuan/Documents/SeqDataSets/Ning1024/2mix_180521s14_100.xls')


In [ ]:
fpath = '/Users/xyuan/Documents/SeqDataSets/Ning1024/12mixtures/19-Table 1.csv'
df_single_19 = load_data(fpath, csv_format=True)

In [ ]:
dfm = match_dfs(df_5mix, df_12mix)
plotly_zone(dfm)
df_12mix.shape, dfm.shape

In [ ]:
df_12mix = pd.read_excel("/Users/xyuan/Documents/SeqDataSets/Ning1024/12mixtures_3'-Biotin_MFE.xls", '12MIX')
df_12mix = thermo_df(df_12mix)

In [ ]:
df_cmc = load_data('/Users/xyuan/Documents/SeqDataSets/Modifications/181227s07_100.xls')

In [ ]:
df_simulated_5mix = load_data('/Users/xyuan/Studio/seq_bank/mix5.csv', csv_format=True)
plotly_zone(df_simulated_5mix)
df_simulated_12mix = load_data('')

In [ ]:
# df_sample = df_2mix[(df_2mix.Mass<8000)&(df_2mix.RT<15)]
# df_sample = df_simulated_5mix.copy()
# df_sample = df_5mix[(df_5mix.Mass<8500)&(df_5mix.RT<10)]
df_sample = df_5mix[(df_5mix.Mass<8500)&(df_5mix.RT<15)]
# df_sample = df_5mix[(df_5mix.Mass<8500)&(df_5mix.RT<10)&(df_5mix.Vol>3E5)]
# df_sample = df_12mix[(df_12mix.RT<15)&(df_12mix.Mass<11000)]
# df_sample = dfm
# df_sample = dft.copy()
# df_sample = df_cmc[(df_cmc.Mass<8000)&(df_cmc.RT<15)].copy()
plotly_zone(df_sample)
plotly_zone(df_sample, y='Vol')

In [ ]:
dft = pd.read_excel('/Users/xyuan/Documents/SeqDataSets/Ning1024/5mix_MFE.xlsx')
dft.shape
# dft = dft[dft.Vol>3E5] # for mix5

In [ ]:

dft = df_12mix[df_12mix.Vol>3E5]
dft.shape

In [ ]:
dft = df_cmc[df_cmc.Vol > 1.5E5]

In [ ]:
def computational_data_seperation(df3p, df5p, full_mass, error=0.1, ignore_endpoints=True, has_tag=False, tag=826.3184):
    df3p_mass_np = np.array(df3p['Mass'])
    df5p_mass_np = np.array(df5p['Mass'])
    mass_sum_np = df3p_mass_np[:, np.newaxis] + df5p_mass_np
    mass_sum_1 = np.round(mass_sum_np, 3)
    mass_sum_df = pd.DataFrame(mass_sum_np)
    mass_sum_df1 = pd.DataFrame(mass_sum_1)

    if has_tag:
        sum_value = round(full_mass + tag + 79.9663 - 18.0106, 1)
        print('full_mass {} sum_value {}'.format(full_mass, sum_value))
    else:
        sum_value = round(full_mass + 18.0106, 1)
        print('full_mass {} sum_value {}'.format(full_mass, sum_value))

    tmp = mass_sum_df1[(mass_sum_df1 >= sum_value-error) & (mass_sum_df1 <= sum_value+error)]
    final_idx_pairs = list(tmp[tmp.notnull()].stack().index)
    df3_idxs = [pair[0] for pair in final_idx_pairs]
    df5_idxs = [pair[1] for pair in final_idx_pairs]
    df3_idxs = list(set(df3_idxs))
    df5_idxs = list(set(df5_idxs))
#     print("df3 {} df5 {}".format(len(df3_idxs), len(df5_idxs)))

    df3p_selected = df3p.iloc[df3_idxs]
    df5p_selected = df5p.iloc[df5_idxs]
    
    if not ignore_endpoints:
        df5p_endpoints = get_end5p_points(df5p, full_mass + 18.0106, tag)
        print("df5p_endpoints {}".format(df5p_endpoints[['Mass', 'RT', 'Vol']]))

        df3p_selected.update(df5p_endpoints)
        df5p_selected.update(df5p_endpoints)
        
    return df3p_selected, df5p_selected

In [ ]:
dfm = match_dfs(df_simulated_5mix, dft)
plotly_zone(dfm)
dfm.shape

In [ ]:

# fullmass = 6600.0153 #6898.0210 # mix2
# fullmass = 7522.0681 #7088.0479 #7079.0513 #6954.9478 #6781.0426 # # # mix5
# fullmass = 6859.019  # mix5 Sulfo Cy3 5'
# fullmass = 6781.0487 #6353.872 # mix12
# fullmass = 6597.119 #6345.922 # cmc
fullmass = 6859.019
df1, _ = computational_data_seperation(df_sample, df_sample, full_mass=fullmass, error=0.1)
plotly_zone(df1)
df1.shape

In [ ]:
bcr = base_calling_random(df1)
plotly_basecalling(*bcr)
plotly_basecalling(df_sample, bcr[1])